# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database


In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of establishments with a hygiene score of 20:", count)

# Display the first document in the results using pprint
result = establishments.find_one(query)
if result:
    print("First document with a hygiene score of 20:")
    pprint(result)
else:
    print("No establishments found with a hygiene score of 20.")


In [ ]:
# Convert the MongoDB query result to a Pandas DataFrame
df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
num_rows = len(df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
num_documents = establishments.count_documents(query)
print("Number of documents:", num_documents)

# Display the first document in the results using pprint
pprint(establishments.find_one(query))


In [ ]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
num_rows = result_df.shape[0]
print("Number of rows:", num_rows)

# Display the first 10 rows of the DataFrame
print(result_df.head(10))


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
degree_search = 0.01
latitude = # 
longitude = # 

# Define the query
query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}

# Define the sort parameter
sort = [("scores.Hygiene", pymongo.ASCENDING)]

# Define the limit parameter
limit = 10  # Example limit, adjust as needed

# Execute the query
results = establishments.find(query).sort(sort).limit(limit)

# Print the results
for result in results:
    pprint(result)


In [ ]:
# Convert result to Pandas DataFrame
result_list = list(results)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(result_list)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline
pipeline = [
    # Match establishments with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
    # Group the matches by Local Authority
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    # Sort the matches from highest to lowest
    {"$sort": {"count": -1}}
]

# Execute the pipeline
result = db.establishments.aggregate(pipeline)

# Print the number of documents in the result
print("Number of documents:", result.count())

# Print the first 10 results
for doc in result.limit(10):
    print(doc)

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result)

# Display the number of rows in the DataFrame
print("Number of rows:", len(df))

# Display the first 10 rows of the DataFrame
print(df.head(10))
